In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import re
from functools import partial
from time import sleep
from datetime import datetime

import sys
import os
sys.path.append(os.path.abspath(".."))
from opt_emergency.api import NaverMap, PublicDataPortal, NearestEr

## nearest_er_xx

In [ ]:
comm_pop_df = pd.read_csv("../data/processed/community_population_df.csv")
emergency_df = pd.read_csv("../data/processed/emergency_df.csv")

tk = ["대구", "경북"]
comm_pop_tk = comm_pop_df[comm_pop_df["시도"].isin(tk)]
emergency_tk = emergency_df[emergency_df["시도"].isin(tk)]

In [ ]:
nearest_er_all = NearestEr(er_df=emergency_tk)
nearest_er_gyjyct = NearestEr(er_df=emergency_tk[emergency_tk["의료기관분류"].str.contains("센터")])

nearest_hpid_list = list()
for row in tqdm(comm_pop_tk.itertuples(), total=comm_pop_tk.shape[0]):
    lon = getattr(row, "경도")
    lat = getattr(row, "위도")
    nearest_info_all = nearest_er_all.get_nearest_er_order_info(lon, lat, N=3)
    nearest_info_gyjyct = nearest_er_gyjyct.get_nearest_er_info(lon, lat)
    nearest_hpid_list.append({
        "nearest_er_hpid_1st": nearest_info_all.loc[0, "hpid"], 
        "nearest_er_travel_time_1st": nearest_info_all.loc[0, "소요시간"],
        "nearest_er_hpid_2nd": nearest_info_all.loc[1, "hpid"], 
        "nearest_er_travel_time_2nd": nearest_info_all.loc[1, "소요시간"],
        "nearest_er_hpid_3rd": nearest_info_all.loc[2, "hpid"], 
        "nearest_er_travel_time_3rd": nearest_info_all.loc[2, "소요시간"],
        "nearest_er_hpid_center": nearest_info_gyjyct["hpid"], 
        "nearest_er_travel_time_center": nearest_info_gyjyct["소요시간"], 
        "측정시각": nearest_info_all.loc[0, "측정시각"]
    })
nearest_info_df = pd.DataFrame(nearest_hpid_list)

In [ ]:
comm_pop_tk.reset_index(drop=True, inplace=True)
comm_pop_tk2 = pd.concat([comm_pop_tk, nearest_info_df], axis=1)
comm_pop_tk2.to_csv("../data/processed/nearest_er_tk_daytime.csv", index=False)

In [ ]:
comm_pop_tk2 = pd.read_csv("../data/processed/nearest_er_tk.csv")
comm_pop_tk2.head(1)

## travel_time_between_er_xx

In [2]:
emergency_df = pd.read_csv("../data/processed/emergency_df.csv")
emergency_df_tk = emergency_df[emergency_df["시도"].isin(["대구", "경북"])]

In [3]:
nm = NaverMap()
hpids = sorted(emergency_df_tk.hpid.tolist())
travel_times = []
for hpid1 in tqdm(hpids):
    for hpid2 in [x for x in hpids if x != hpid1]:
        pos_start = emergency_df_tk.loc[emergency_df_tk.hpid == hpid1, ["경도", "위도"]].values[0]
        pos_goal = emergency_df_tk.loc[emergency_df_tk.hpid == hpid2, ["경도", "위도"]].values[0]
        tt = nm.get_travel_time(pos_start, pos_goal)
        travel_times.append([hpid1, hpid2, tt])

  0%|          | 0/56 [00:00<?, ?it/s]

In [4]:
df = pd.DataFrame(travel_times, columns=["hpid1", "hpid2", "travel_time"])
df["측정시각"] = datetime.now().strftime("%y%m%d_%H%M%S")
df.to_csv("../data/processed/travel_time_between_er_tk_daytime.csv", index=False)